# Installing required packages

In [22]:
!pip install mysql-connector-python

# Importing the packages

In [23]:
import pandas as pd
import numpy as np
import os
import csv
from pandas import ExcelWriter
import os.path as path
import mysql.connector
from mysql.connector import Error
import logging

# Entering the file location 

In [46]:
source_loc = input("Enter the file location where all the text file is present")

Enter the file location where all the text file is presentC:/Users/srika/Desktop/corteva


# Reading the file Name

In [47]:
#file_type = ".txt"
direc = os.listdir(source_loc)

### Function for removing .txt from files name so that we can get station code from txt file name

In [48]:
def removedottxt(str):
    newstr =""
    for i in str:
        if (i=='.'):
            return newstr;
        newstr = newstr + i 
        


### Function for changing the date formate YYYYMMDD to YYYY-MM-DD

In [49]:
def converIntoDataFormate(inputString):
    inputString = inputString = inputString[:4] + '-' + inputString[4:6] + '-' + inputString[6:]
    return inputString

### function for adding NULL in the query where data is not present

In [50]:
def removingUndefinedValue(str):
    return str.replace("None", "Null")

### Function for creating the table which has average values

In [51]:
def CreateStatTableForStation(stationcode):
    try:
        con = mysql.connector.connect(host="localhost",user="root",password="toor",database="weatherdataapp",charset="utf8")
        cur = con.cursor()
        query = "select year(DT) from weather where StationCode = '"+stationcode+"' group by year(DT);"
        cur.execute(query)
        data = cur.fetchall()
        con.commit()
        for year in data:
            query = "select AVG(MaxTemp),AVG(MinTemp),sum(precipitation) from weather where year(DT)= "+str(year[0])+" AND StationCode = '"+stationcode+"';"
            cur.execute(query)
            avgdata = cur.fetchall()
            query = "insert into weatherstat (year,StationCode,AverageMaxTemp,AverageMinTemp,Totalprecipitation)values("+str(year[0])+",'"+stationcode+"',"+str(avgdata[0][0])+","+str(avgdata[0][1])+","+str(avgdata[0][2])+")"
            cur.execute(query)
        con.commit()
        con.close()
    except Error as e:
        print(e)
    finally:
        if(con.is_connected()):
            con.close()  

### for Keeping track of  the how many files is uploaded and how many is left

In [52]:
totalcount = 0
for file in direc:
    if file.endswith(".txt"):
       totalcount = totalcount+1 

### Function for sending data in the database 

In [53]:
def dataframetosql(dataframe,station_code):
    try:
        con = mysql.connector.connect(host="localhost",user="root",password="toor",database="weatherdataapp",charset="utf8")
        cur = con.cursor()
        query = "select StationCode from weather where StationCode = '"+ station_code+ "' Limit 1;"
        cur.execute(query)
        data = cur.fetchone()
        for(row,rs) in dataframe.iterrows():
            date = converIntoDataFormate(str(rs[0]))
            maxtemp = str(rs[1])
            mintemp = str(rs[2])
            prep = str(rs[3])
            if (data):
                logging.basicConfig(filename="log.txt", level=logging.DEBUG)
                log_string = "data is already inserted for station Code : "+ station_code + ": please put data merg.csv and run mergData function"
                logging.debug(log_string)
                break;
            query = "insert into weather (StationCode,DT,MaxTemp,MinTemp,precipitation) values("+"'"+station_code+"',"+ "'"+date+"'"+","+maxtemp+","+mintemp+","+prep+");"
            query = removingUndefinedValue(query)
            cur.execute(query)
        con.commit()
        if(data):
            print("Already Uploaded")
            print("---------------------------------------------------------------------")
        else:
            log_string = "No of record inserted : "+ str(df.shape[0]) + ", for Station Code : "+ station_code
            logging.basicConfig(filename="log.txt", level=logging.DEBUG)
            logging.debug(log_string)
            CreateStatTableForStation(station_code)
            print("Uploaded")
            print("---------------------------------------------------------------------")
    except Error as e:
        print(e)
    finally:
        if(con.is_connected()):
            con.close()     

### funtion for reading the file converting into dataframe and call above function

In [55]:
count = 1
print ("Total station :",totalcount)
for file in direc:
    if file.endswith(".txt"):
        #source_loc = source_loc + file
        header = ["date","Max","Min","prep"]
        station_code = removedottxt(file);
        if (station_code=='log'):
            continue
        print(file)
        df = pd.read_csv(file , encoding="utf-8",delimiter='\t',names=header)
        df.replace(-9999,None,inplace =True)
        
        leftcount = totalcount-count
        print("data uploading is start in database for station code : ",station_code)
        print("Total station left :",leftcount)
        print("uploading........")
        dataframetosql(df,station_code);
        count = count+1

Total station : 23
USC00110072.txt
data uploading is start in database for station code :  USC00110072
Total station left : 22
uploading........
Already Uploaded
---------------------------------------------------------------------
USC00110187.txt
data uploading is start in database for station code :  USC00110187
Total station left : 21
uploading........
Already Uploaded
---------------------------------------------------------------------
USC00110338.txt
data uploading is start in database for station code :  USC00110338
Total station left : 20
uploading........
Already Uploaded
---------------------------------------------------------------------
USC00111280.txt
data uploading is start in database for station code :  USC00111280
Total station left : 19
uploading........
Already Uploaded
---------------------------------------------------------------------
USC00111436.txt
data uploading is start in database for station code :  USC00111436
Total station left : 18
uploading........
Alr

# run below script when we have to update the database

#### create a  folder named merg where all text file is present and put the update text file in that folder

In [62]:
def mergedata(source_loc):
    file_type = ".txt"
    direc = os.listdir(source_loc)
    for file in direc:
        if file.endswith(".txt"):
            header = ["date","Max","Min","prep"]
            datafile = source_loc+"/"+ file
            df = pd.read_csv(datafile , encoding="utf-8",delimiter='\t',names=header)
            df.replace(-9999,None,inplace =True)
            station_code = removedottxt(file);
            for(row,rs) in df.iterrows():
                updateToDatabase(station_code,converIntoDataFormate(str(rs[0])),str(rs[1]),str(rs[2]),str(rs[3]))
            
def updateToDatabase(stationcode,date,maxtemp,mintemp,prep):
    try:
        con = mysql.connector.connect(host="localhost",user="root",password="toor",database="weatherdataapp",charset="utf8")
        cur = con.cursor()
        query = "UPDATE weather SET  MaxTemp = "+maxtemp+", MinTemp = "+mintemp+", precipitation = "+prep+" where StationCode = '"+stationcode+"'and DT = '"+date+"';"
        cur.execute(query)
        query = "select AVG(MaxTemp),AVG(MinTemp),sum(precipitation) from weather where year(DT)= "+date[0:4]+" AND StationCode = '"+stationcode+"';"
        cur.execute(query)
        avgdata = cur.fetchone()
        query = "update weatherstat set AverageMaxTemp = "+str(avgdata[0])+", AverageMinTemp = "+str(avgdata[1])+", Totalprecipitation = "+str(avgdata[2])+" where StationCode = '"+stationcode+"' and year = '"+date[0:4]+"';"
        cur.execute(query) 
        con.commit()
    except Error as e:
        print(e)
    finally:
        logging.basicConfig(filename="log.txt", level=logging.DEBUG)
        log_string = "data is updated for station Code : "+ station_code
        logging.debug(log_string)
        print(log_string)
        if(con.is_connected()):
            con.close()
mergedata(input("Enter the file location where update data is present"))

Enter the file location where update data is presentC:/Users/srika/Desktop/corteva/merg
data is updated for station Code : USC00335315
